In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

import sys

sys.path.append('/content/drive/My Drive')

Mounted at /content/drive


In [2]:
!pip install datasets
!pip install sacremoses
!pip install evaluate
!pip install rouge_score
# !pip install --upgrade torch torchvision torchaudio

In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
from transformers import MarianTokenizer
from torch.utils.data import Dataset, DataLoader
from datasets import load_dataset

from model import NMTModel, Trainer
from dataloader import NMTDataset

In [4]:
dataset = load_dataset('cfilt/iitb-english-hindi')

# dataset = dataset.train_test_split(test_size=0.1)
train_data = dataset['train']
train_data = train_data.select(range(250000))
val_data = dataset['validation']

model_name = "Helsinki-NLP/opus-mt-en-hi"
tokenizer = MarianTokenizer.from_pretrained(model_name)

max_length = 100

train_dataset = NMTDataset(train_data, tokenizer, tokenizer, max_length)
val_dataset = NMTDataset(val_data, tokenizer, tokenizer, max_length)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [5]:
model = NMTModel(model_name)
trainer = Trainer(model, train_dataset, val_dataset, epochs = 3, batch_size=64, lr=3e-5)

In [6]:
# from transformers import MarianMTModel
# model = MarianMTModel.from_pretrained("Helsinki-NLP/opus-mt-en-hi")  # replace with your model
# num_decoder_layers = model.config.decoder_layers

In [11]:
trainer.train()

Epoch 1/3: 100%|██████████| 3907/3907 [39:58<00:00,  1.63batch/s, loss=0.384]


Epoch 1/3, Training Loss: 0.8210143589106916
Validation Loss: 2.3380718098746405


Epoch 2/3: 100%|██████████| 3907/3907 [39:48<00:00,  1.64batch/s, loss=0.63]


Epoch 2/3, Training Loss: 0.5819060262097158
Validation Loss: 2.228149665726556


Epoch 3/3: 100%|██████████| 3907/3907 [39:50<00:00,  1.63batch/s, loss=0.454]


Epoch 3/3, Training Loss: 0.4993439003470489
Validation Loss: 2.1607046524683633
Training Complete.


In [6]:
model = NMTModel(model_name)
model_path = "/content/drive/My Drive/ft_marian.pt"
model.load_state_dict(torch.load(model_path, map_location=torch.device('cpu')))

<All keys matched successfully>

In [7]:
from evaluate_models import calculate_scores

In [8]:
calculate_scores(model, tokenizer, tokenizer, 100)

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
Evaluating:   3%|▎         | 68/2507 [04:34<2:44:12,  4.04s/it]


KeyboardInterrupt: 